# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [4]:
# Read the WeatherPy_vacation.csv into a DataFrame
csv_path = "../Vacation_Search/WeatherPy_vacation.csv"
vacation_df = pd.read_csv(csv_path)

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Punta Arenas,CL,53.71,clear sky,-53.1500,-70.9167,Hotel Lacolet
1,17,Teacapan,MX,68.05,broken clouds,22.5500,-105.7500,illas y Hotel Maria Fernanda
2,22,Awbari,LY,52.57,clear sky,26.5921,12.7805,فندق اوباري
3,23,Castro,BR,60.01,overcast clouds,-24.7911,-50.0119,Hotel Central Palace
4,26,Hermanus,ZA,63.59,light rain,-34.4187,19.2345,Aloe guest house


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [5]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Cape Town"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Cape Town"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Springbok"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Upington"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Port Elizabeth"]
itinerary = [vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [7]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat(itinerary, ignore_index=True)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,38,Cape Town,ZA,65.10,few clouds,-33.9258,18.4232,Townhouse Hotel
1,432,Springbok,ZA,52.93,scattered clouds,-29.6643,17.8865,Elkoweru Guesthouse
2,543,Upington,ZA,64.78,clear sky,-28.4478,21.2561,Die Eiland Holiday Ressort
3,111,Port Elizabeth,ZA,64.69,clear sky,-33.9180,25.5701,Waterford Hotel
4,38,Cape Town,ZA,65.10,few clouds,-33.9258,18.4232,Townhouse Hotel



## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [8]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lng","Lat"]].copy()

# Display sample data
waypoints_df

,Lng,Lat
0,18.4232,-33.9258
1,17.8865,-29.6643
2,21.2561,-28.4478
3,25.5701,-33.9180
4,18.4232,-33.9258


## Use GeoViews to create map that shows the four cities in the itinerary

In [9]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 500,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

In [10]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Current Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [11]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [12]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '9060c3e83a9745c7a61dbfd047182a30',
 'waypoints': '-33.9258,18.4232|-29.6643,17.8865|-28.4478,21.2561|-33.918,25.5701|-33.9258,18.4232'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [13]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [14]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [15]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in np.arange(4):
    for row in legs[leg]:
        longitude.append(row[0])
        latitude.append(row[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [16]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head()

,longitude,latitude
0,18.422919,-33.926062
1,18.423419,-33.926432
2,18.424143,-33.926975
3,18.424191,-33.927011
4,18.424246,-33.926958


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [17]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df).options(line_color='red', line_width=2)

In [18]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]